In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import adam_v2
from keras.metrics import RootMeanSquaredError

# 0. Daten vorbereiten

Bevor wir mit unseren Daten arbeiten können, müssen wir diese erst einmal vorbereiten. Darunter fällt das **Einlesen**, das **Vereinheitlichen von Timestamps und Fußgänger-IDs**, das **Berechnen der Delta-Werte** für die x- und y-Position zwischen zwei Zeitschritten, sowie das **Aufteilen in Trainings- und Testdaten**.

In [ ]:
path_train = "Data/datasplits/traindata.csv"
path_test = "Data/datasplits/testdata.csv"

# Datenvorbereitung wird nur einmal ausgeführt; Sobald Trainings- und Testdaten als .csv Dateien abgespeichert wurden, werden diese einfach nur eingelesen
# Um einen neuen Trainings-/Testsplit zu erhalten, einfach die csv-Dateien aus dem Ordner "Data/datasplits" entfernen
if not os.path.exists(path_train) or not os.path.exists(path_test):
    # Eine genauere Beschreibung, was bei der Vorbereitung der Daten passiert, ist in der Datei "DataPreparation.ipynb" selbst zu finden
    %run DataPreparation.ipynb

# Einlesen von Trainings- und Testdaten
df_train = pd.read_csv(path_train, sep=",")
df_test = pd.read_csv(path_test, sep=',')

In [ ]:
df_train

# 1. Daten vorverarbeiten

Durch die ersten 8 Werte erhalten wir 7 Deltas. Der erste Wert ist immer 0 (da wir ja keinen vorherigen Wert haben, um das Delta zu berechnen). Aus diesem Grund wird der erste Wert entfernt und dann später nur die sieben Delta-Werte in das NN gegeben.

In [ ]:
# Alle ersten Zeitschritte werden entfernt (da dx und dy hier eh immer 0 ist)
df_train = df_train.drop(df_train[df_train.Timestamp == 1].index)

## 1.3 Standardisieren

In [ ]:
# Trainings- und Validierungsdaten Daten skalieren
scaler = MinMaxScaler(feature_range = (-1,1)) # = StandardScaler() oder RobustScaler()
features = df_train[["dx", "dy"]]
scaler.fit(features)
df_train[["dx_scaled", "dy_scaled"]] = scaler.transform(features)

In [ ]:
df_train

Nun werden die Trainingsdaten so vorbereitet, dass man sie in das RNN einfüttern kann. Dazu brauchen sie die das Format [samples, time steps, features]. Nachdem wir später eine rekursive Single-Step Prediction für den nächsten Delta-Wert (basierend auf den 7 vorhergehenden) durchführen werden, muss man die Daten zusätzlich noch weiter aufteilen, was auch den Vorteil hat, dass man mehr Trainingsdaten hat. Jeder Datenpunkt wird in weitere Datenpunkte aufgeteilt, bestehend aus 7 Werten und der 8e Wert ist dann die erwartete Prediction

In [ ]:
sequence_length = 7 # Anhand von 7 dx und dy Werten (welche aus 8 Zeitschritten entstanden sind) sagen wir den nächsten Delta-Wert voraus

def transform_data_for_rnn(df, sequence_length):
    ids = np.array(df.ID.unique()) # alle (20 Timestep) Sequenzen aus dem dataframe holen, um die dann weiter aufzuteilen
    x, y = [], []
    for id in ids:
        df_current = df[df.ID == id] # einen Datenpunkt mithilfe der ID herausgreifen und diesen dann in weitere Datenpunkte aufteilen
        feature_data = np.array(df_current[['dx_scaled', 'dy_scaled']])
        for i in range(sequence_length, feature_data.shape[0]):
            x.append(feature_data[i-sequence_length:i,:]) #contains sequence_length values 0-sequence_length * columsn
            y.append(feature_data[i, :]) #contains the prediction values for validation (3rd column = Close),  for single-step prediction
    x = np.array(x)
    y = np.array(y)
    return x, y

x_train, y_train = transform_data_for_rnn(df_train, sequence_length)

In [ ]:
print(x_train.shape, y_train.shape)

# 2. Neuronales Netz erstellen und trainieren

Zunächst erstellen wir unser NN mit all seinen Layern und Eigenschaften.

In [ ]:
model = Sequential()
# input shape = 7 (Timeseries) * 2 (features, dx, dy)
model.add(LSTM(units = 14, dropout = 0.2, return_sequences = False, input_shape = (x_train.shape[1], x_train.shape[2]))) 
#model.add(LSTM(units = 14, dropout = 0.2, return_sequences = False)) 
#model.add(Dense(units = 7))
model.add(Dense(units = 2)) # default activation function = 'linear'
print(model.summary())

Jetzt wird das Model konfiguriert und mithilfe der Trainingsdaten trainiert. 

In [ ]:
opt = adam_v2.Adam(learning_rate=0.0001) # decay=lr/epochs)
model.compile(loss = 'mse', optimizer = opt)#, metrics = [RootMeanSquaredError()])
history = model.fit(x = x_train, y = y_train, epochs = 30, validation_split = 0.1, batch_size = 12, shuffle = True)

Der Verlauf der Loss-Funktion sieht folgendermaßen aus...

In [ ]:
def plot_model_loss(history, epochs):
    fig, ax = plt.subplots(figsize=(5, 5), sharex=True)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("Model loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    ax.xaxis.set_major_locator(plt.MaxNLocator(epochs))
    plt.legend(["Training", "Validation"], loc="best")
    plt.grid()
    print(history.history.keys())
    plt.show()

def plot_model_metric(history, epochs):
    fig, ax = plt.subplots(figsize=(5, 5), sharex=True)
    plt.plot(history.history['root_mean_squared_error'])
    plt.plot(history.history["val_root_mean_squared_error"])
    plt.title("Mean Squared Error")
    plt.ylabel("MSE")
    plt.xlabel("Epoch")
    ax.xaxis.set_major_locator(plt.MaxNLocator(epochs))
    plt.legend(["Training_MSE", "Validation_MSE"], loc="best")
    plt.grid()
    plt.show()

In [ ]:
plot_model_loss(history, 5)
#plot_model_metric(history, 5)

# 3. Vorhersage von zukünftigen Deltas/ Positionen

In [ ]:
current_timestep = df_test[df_test.ID == 5] # Zeitschritt mit aktueller ID holen
d = current_timestep.loc[(current_timestep.Timestamp > 1) & (current_timestep.Timestamp < 9)] # Startdeltas extrahieren (2-8)
print(d)
#prediction_values = np.array(d[['dx', 'dy']]) 
prediction_values = np.array(scaler.transform(d[['dx', 'dy']])) # skalieren der Daten + Umwandeln in Numpy-Array
print(prediction_values)
prediction_values = np.reshape(prediction_values, (1, 7, 2)) # Startdeltas formatieren, sodass man sie ins NN fütter kann
print(prediction_values)
# model.predict(prediction_values) # neues Delta predicten
prediction_values = np.reshape(prediction_values, (7,2))
prediction_values = np.delete(prediction_values, 0, axis=0) # ersten Wert der gegebenen Werte entfernen
prediction_values = np.append(prediction_values, np.array([[2 , 1.4]]) , axis=0) # predicteten Wert für nächste Vorhersage anhängen
print(np.reshape(prediction_values, (1, 7, 2)))

In [ ]:
# verschiebt die Werte des Arrays um eins nach vorne und hängt den vorhergesagten Delta-Wert hinten an (erster Wert wird entfernt)
# Prinzip: aus [1, 2, 3, 4] und [5] wird [2, 3, 4, 5]
# current_deltas braucht Shape (1, 7, 2) und predicted_deltas (1, 2)
def shift_deltas(current_deltas, predicted_delta):
    new_deltas = np.reshape(current_deltas, (7,2))
    new_deltas = np.delete(new_deltas, 0, axis=0) # ersten Wert der gegebenen Werte entfernen
    new_deltas = np.append(new_deltas, predicted_delta , axis=0) # predicteten Wert für nächste Vorhersage anhängen
    return np.array([new_deltas])

# given_deltas = start_deltas = die Deltas der ersten 8 Positionen, mit denen die nächsten n Positionen vorhergesagt werden sollen
# x_pos = x-Koordinaten Startwert = letzter gemessener x-Wert, bevor die Prädiktion startet
# y_pos = y-Koordinaten Startwert = letzter gemessener y-Wert, bevor die Prädiktion startet
def predict_next_n_steps(n, given_deltas, x_pos, y_pos):
    pred_dx_values = [] # vorhergesagte Delta-x Werte
    pred_dy_values = [] # vorhergesagte Delta-y Werte
    pred_x_pos = [] # vorhergesagte x Koordinaten
    pred_y_pos = [] # vorhergesagte y Koordinaten  
    for i in range(0, n): 
        pred_value = model.predict(given_deltas) # nächsten Wert anhand der aktuellen 7 Deltas vorhersagen
        pred_value_unscaled = np.round(scaler.inverse_transform(pred_value), 3) # Ergebnis zurückskalieren, um tatsächliche Delta-Werte zu erhalten
        given_deltas = shift_deltas(given_deltas, pred_value) # Vorhergesagten Delta-Wert zur Liste für die nächste Prädiktion hinzufügen
        pred_dx = pred_value_unscaled[0, 0] # herausholen des vorhergesagten dx Wertes aus dem Array
        pred_dy = pred_value_unscaled[0, 1] # herausholen des vorhergesagten dy Wertes aus dem Array
        x_pos = round(x_pos + pred_dx, 3)
        y_pos = round(y_pos + pred_dy, 3)
        # berechneten Werte für dx, dy, xPos und yPos an die jeweilige Ergebnisliste anhängen
        pred_dx_values.append(pred_dx)
        pred_dy_values.append(pred_dy)
        pred_x_pos.append(x_pos)
        pred_y_pos.append(y_pos)
    return pred_dx_values, pred_dy_values, pred_x_pos, pred_y_pos

pred_dx_values, pred_dy_values, pred_x_pos, pred_y_pos = predict_next_n_steps(12, lala, 0, 0)

In [47]:
forecast_range = 12 # wir wollen die nächsten 12 Positionen vorhersagen

predicted_dx_values = []
predicted_dy_values = []
predicted_x_pos = []
predicted_y_pos = []

ids =  np.array(df_test.ID.unique())

for id in ids: 
    # Aktuellen Datenpunkt mithilfe der ID auswählen
    current_datapoint = df_test[df_test.ID == id]
    given_data = current_datapoint.loc[current_datapoint.Timestamp <= 8] # ersten 8 Positionen sind gegeben, die nächsten 12 sollen vorhergesagt werden
    
    # anhängen der ersten 8 gegebenen Werte an die Trajektorien-Resultat-Listen
    predicted_dx_values.extend(given_data['dx'].tolist())
    predicted_dy_values.extend(given_data['dy'].tolist())
    predicted_x_pos.extend(given_data['X'].tolist())
    predicted_y_pos.extend(given_data['Y'].tolist())
    
    # von den gegebenen 8 Punkten die 7 Deltas extrahieren und basierend auf diesen die Vorhersagen machen
    df_train.drop(df_train[df_train.Timestamp == 1].index)
    given_deltas = given_data.drop(given_data[given_data.Timestamp == 1].index)
    given_deltas = np.array(scaler.transform(given_deltas[['dx', 'dy']])) # skalieren der Daten + Umwandeln in Numpy-Array
    given_deltas = np.reshape(given_deltas, (1, 7, 2))
    
    # Positionen des Fußgängers holen, an der er sich am Ende der beobachteten Zeit befindet --> = Startkoordinaten für die Prädiktion
    start_x = predicted_x_pos[-1]
    start_y = predicted_y_pos[-1]
    
    # nächsten Positionen vorhersagen
    predicted_dx, predicted_dy, pred_x, pred_y = predict_next_n_steps(forecast_range, given_deltas, start_x, start_y)
    break

# Erstellen Listen als Spalten an das Test-Dataframe anhängen
#df_test["predicted_dx_values"] = predicted_dx_values
#df_test["predicted_dy_values"] = predicted_dy_values
#df_test["predicted_x_pos"] = predicted_x_pos
#df_test["predicted_y_pos"] = predicted_y_pos

[[[-0.18006841 -0.21618316]
  [-0.18000134 -0.21520474]
  [-0.18127557 -0.21618316]
  [-0.18000134 -0.21148672]
  [-0.18006841 -0.21148672]
  [-0.18516531 -0.21148672]
  [-0.18000134 -0.21148672]]]
8
8


In [46]:
df_test

,Timestamp,ID,X,Y,dx,dy,lala
0,1,5,-4.338,17.564,0.000,0.000,None
1,2,5,-4.262,17.468,0.076,-0.096,None
2,3,5,-4.185,17.392,0.077,-0.076,None
3,4,5,-4.127,17.296,0.058,-0.096,None
4,5,5,-4.050,17.296,0.077,0.000,None
...,...,...,...,...,...,...,...
10795,16,5399,-18.102,19.270,-0.449,0.000,None
10796,17,5399,-18.618,19.270,-0.516,0.000,None
10797,18,5399,-19.068,19.270,-0.450,0.000,None
10798,19,5399,-19.494,19.270,-0.426,0.000,None


In [ ]:
    current_timestep = df_test[df_test.ID == id] # Zeitschritt mit aktueller ID holen
    deltas = current_timestep.loc[(current_timestep.Timestamp > 1) & (current_timestep.Timestamp < 9)] # Initialen Delta-Werte extrahieren (die ersten 7 Deltas)
    
    given_deltas = np.array(scaler.transform(d[['dx', 'dy']])) # skalieren der Daten + Umwandeln in Numpy-Array
    given_deltas = np.reshape(given_deltas, (1, 7, 2)) # Startdeltas formatieren, sodass man sie ins NN fütter kann
    
    
    for i in range(0, forecast_range):
        
        predicted_deltas = model.predict(current_timesteps)
        # skalierten Wert zu den Vorhersagewerten hinzufügen
        predicted_deltas = scaler.inverse_transform(predicted_deltas) # zurückskalieren der Werte, um die eingentlichen Deltas zu erhal

# Plotting

In [ ]:
def plot_pedestrian_trajectory(x_observed, y_observed, x_pred, y_pred, x_real = np.array([]), y_real = np.array([])):
    # Letzten Wert der beobachteten Position an das Array der vorhergesagten Positionen anhängen, sodass im Plot eine zusammenhängende Trajektorie dargestellt werden kann
    x_pred = np.insert(x_pred, 0, x_observed[-1], axis=0)
    y_pred = np.insert(y_pred, 0, y_observed[-1], axis=0)
    if x_real.size != 0 and y_real.size != 0:
        x_real = np.insert(x_real, 0, x_observed[-1], axis=0)
        y_real = np.insert(y_real, 0, y_observed[-1], axis=0)
        plt.plot(x_real, y_real, label = "Real future positions", color='green', linestyle='dotted', linewidth = 1,
             marker='o', markerfacecolor='green', markersize=5)
    # Plotten der vorhergesagten Positionen
    plt.plot(x_pred, y_pred, label = "Predicted future positions", color='red', linestyle='dotted', linewidth = 1,
             marker='o', markerfacecolor='red', markersize=5)
    # Plotten der beobachteten Positionen
    plt.plot(x_observed, y_observed, label = "Observed positions", color='blue', linestyle='dotted', linewidth = 1,
             marker='o', markerfacecolor='blue', markersize=5)
    plt.legend(loc='best')
    plt.xlabel('x - coordinate')
    plt.ylabel('y - coordinate')
    plt.title('Trajectorie of a single pedestrian')
    plt.show()

In [ ]:
# Beispiel-Plot
gegeben_x = np.array([1,2,3,4])
gegeben_y = np.array([1,1,1,1])
pred_x = np.array([5,6,7,8,7,6])
pred_y = np.array([1,1,1,1,2,2])
real_x = np.array([5,6,7,7])
real_y = np.array([2,3,4,4])
plot_pedestrian_trajectory(gegeben_x, gegeben_y, pred_x_pos, pred_y_pos, x_real = real_x, y_real = real_y)

# Normalize and standardize data
You can normalize your dataset using the scikit-learn object MinMaxScaler.

Good practice usage with the MinMaxScaler and other rescaling techniques is as follows:

Fit the scaler using available training data. For normalization, this means the training data will be used to estimate the minimum and maximum observable values. This is done by calling the fit() function,
Apply the scale to training data. This means you can use the normalized data to train your model. This is done by calling the transform() function
Apply the scale to data going forward. This means you can prepare new data in the future on which you want to make predictions.
If needed, the transform can be inverted. This is useful for converting predictions back into their original scale for reporting or plotting. This can be done by calling the inverse_transform() function.
https://machinelearningmastery.com/normalize-standardize-time-series-data-python/

# k-Fold Cross Validation with validation set

In general, when you are doing model selection and testing, your data is divided into three parts, training set, validation set and testing set. You use your training set to train different models, estimate the performance on your validation set, then select the model with optimal performance and test it on your testing set.

On the other hand, if you are using K-fold cross-validation to estimate the performance of a model, your data is then divided into K folds, you loop through the K folds and each time use one fold as testing(or validation) set and use the rest (K-1) folds as training set. Then you average across all folds to get the estimated testing performance of your model. This is what the Wikipedia page is referring to.

But keep in mind that this is for testing a specific model, if you have multiple candidate models and want to do model-selection as well, you have to select a model only with your training set to avoid this subtle circular logic fallacy. So you further divide your (K-1) folds 'training data' into two parts, one for training and one for validation. This means you do an extra 'cross-validation' first to select the optimal model within the (K-1) folds, and then you test this optimal model on your testing fold. In other words, you are doing a two-level cross-validation, one is the K-fold cross-validation in general, and within each cross-validation loop, there is an extra (K-1)-fold cross-validation for model selection. Then you have what you stated in your question, 'Of the k subsamples one subsample is retained as the validation data, one other subsample is retained as the test data, and k-2 subsamples are used as training data.'
https://stats.stackexchange.com/questions/90288/in-k-fold-cross-validation-does-the-training-subsample-include-test-set

## Why separate test and validation sets?
The error rate estimate of the final model on validation data will be biased (smaller than the true error rate) since the validation set is used to select the final model After assessing the final model on the test set, YOU MUST NOT tune the model any further!
https://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set

# MeanAbsoluteError und MeanSquaredError Test
https://keras.io/api/losses/regression_losses/

In [ ]:
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError
y_true = [[0.5, 2]]
y_pred = [[0.5, 1.5]]
# Using 'auto'/'sum_over_batch_size' reduction type.
mae = MeanAbsoluteError()
print(mae(y_true, y_pred).numpy())